In [26]:
import io

#Excepts a filename that has the exlusion words, parses it and adds it to a set 
def excludeList(filename):
    if len(filename) < 1:
        filename = 'defaultStopWords.txt'
    setOfExcludedWords = set()
    with io.open(filename, 'r',encoding='utf-8', errors='ignore') as inputFile:
        for line in inputFile:
            words = line.split()
            for word in words:
                setOfExcludedWords.add(word)
    print("Excluding the following words:")
    print(setOfExcludedWords)
    return setOfExcludedWords

In [27]:
def addBackCases(word):
    #Start with the last three letters because dont want to accidentally just add an 'A' when it should be 'AZ/R'
    lettersThree = word[-3:]
    
    #Remove suffix add 'AZ'
    if(lettersThree == 'PZL'):
        l = []
        word = word[0:-3]
        l.append(word)
        l.append('AZ')
        s = ''.join(l)
        return s
    #Remove suffix add 'R'
    if(lettersThree == 'EZL'):
        l = []
        word = word[0:-3]
        l.append(word)
        l.append('R')
        s = ''.join(l)
        return s
    #Remove suffix add 'A'
    lettersTwo = word[-2:]
    if(lettersTwo == 'ZL'):
        l = []
        word = word[0:-2]
        l.append(word)
        l.append('A')
        s = ''.join(l)
        return s
    #Ends in just the letter 'L' with no need to add back letters so just remove the last letter
    else:
        word = word[0:-1]
        return word

In [28]:
def transformWord(word):
    endLetter = word[-1]
    if(endLetter == 'L'):
        #Four scenarios if the character ends in L --> just remove L, 'ZL', 'PZL', and 'EZL' cases
        word = addBackCases(word)
        return word
    # Check the last two letters -> 'LZ' vs 'ZQ'
    endLetter2 = word[-2:]
    if(endLetter2 == 'LZ' or endLetter2 == 'ZQ'):
        #remove the two letters
        word = word[0:-2]
        return word
    # Check the last three letters -> 'EVM'
    endLetter3 = word[-3:]
    if(endLetter3 == 'EVM'):
        #remove the last three letters
        word = word[0:-3]
        return word
    else:
        return word

In [43]:
fname = input('Enter file: ')
excludeStopWords = input('Exclude stop words(Y/N): ')
extractStemWords = input('Consider stem words(Y/N): ')
topCount = input('Enter the top results desired: ')
topCount = int(topCount)
stopWordSet = {}
#Determine if stop words need to be considered. If Yes then exclude but by default we will incorporate stem words 
if(excludeStopWords.lower() == 'y' or excludeStopWords.lower() == 'yes'):
    excludeBoolean = True
    excludeFileName = input('Which file contains the exclude stop words: ')
    stopWordSet = excludeList(excludeFileName)
else:
    excludeBoolean = False

    #Determine if stem words need to be considered 
if(extractStemWords.lower() == 'y' or extractStemWords.lower() == 'yes'):
    extractBoolean = True
else:
    extractBoolean = False


Enter file: 
Exclude stop words(Y/N): y
Consider stem words(Y/N): y
Enter the top results desired: 25
Which file contains the exclude stop words: 
Excluding the following words:
{'KMDK', 'KH', 'EK', 'HA', '??', 'DVQ', 'YVEAZ', 'D', 'KMZEU', 'EV', 'KMZ', 'AHU'}


In [44]:
import io
import os

#If no input let's default to the exercise document 
if len(fname) < 1 : fname = 'exercisedocument.txt'

file = open(fname)

dictionary = dict()

    
originalText = []

#Read the file and encode it to utf-8 since step one says to ignore non-letter characters in the text file.
#Using an import called io to help with input output and using with open we do not have to worry about closing the file
#since it will be taken care of after the reading is done
with io.open(fname, 'r',encoding='utf-8', errors='ignore') as inputFile:
    for line in inputFile:
        originalText.append(line)
        words = line.split()
        
        for word in words:
            #If excluding stop words then do not add it to frequency count 
            if(excludeBoolean == True):
                if(word in stopWordSet):
                    continue
            #Adding filtered/altered/untouched word to dictionary (3 cases)
            if(extractBoolean == True):
                #Extract the stem so transform the word
                word = transformWord(word)
            if(word in dictionary):
                dictionary[word] = dictionary[word] + 1
            else:
                dictionary[word] = 1


#sorted is built in to sort dictionaries, lists, etc 
#we are specifically sorting by v[1] which is the value in the dictionary or index 2 of an index
#reverse = true to do descending order otherwise it defaults to ascending order 
sortedByVal = {k:v for k,v in sorted(dictionary.items(), key= lambda v:v[1], reverse = True)}


counter = 0
#Write to a new file and overwrite anything that exists by appending the desired number to file name 
newFileName = 'top_' + str(topCount)+ "_" + fname

#If file exists already that we will create a new file but append a number 
if(os.path.exists(newFileName)):
    fileNameCount = 1
    #While it exists let's increment the number until we find an instance where it is unique
    while(os.path.exists(newFileName)):
        newFileName = 'top_' + str(topCount)+ "_" + fname
        newFileName = newFileName + '(' + str(fileNameCount) +')'
        fileNameCount = fileNameCount + 1

#Save the contents into this new file 
with io.open(newFileName, mode='w') as f:
    for k,v in sortedByVal.items():
        #Manual counter to show the top results from the sorted dictionary
        if counter < topCount:
            text = str(counter+1)+ ') '+ k + ' ' + 'Count: ' + str(v) + '\n'
            f.write(text)
            counter = counter + 1
        #Exit the for loop when the condition is met (wrote the number of results we wanted)
        else:
            break

#Append original text, stop words setting, and resulting word frequencies 

originalText = ''.join(originalText)

with io.open(newFileName, mode='a') as f:
    f.write('\n')
    f.write("List of Stop Words: \n")
    for setVal in stopWordSet:
        f.write(setVal + '\n')


    f.write('\n')
    f.write('Original Text\n')
    f.write(originalText)

        